# Bluesky sample stage operation

Cleaning up bluesky routines and testing sample stage scan before commissioning

In [2]:
# October 2024
# awojdyla@lbl.gov, xchong@lbl.gov, francisho@lbl.gov

In [3]:
import epics
import bluesky
#import numpy
#import matplotlib.pyplot as plt
#import time 

In [4]:
from bluesky import RunEngine
RE = RunEngine({})

In [5]:
from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

0

### Connecting to EPICS motors
We had some issues here witht the picoscale interformeter -- we need to use caproto to talk to it (ophyd's motor does not work directly out of the box)

from: https://nsls-ii.github.io/bluesky/tutorial.html

```
from ophyd import EpicsMotor
nano_top_x = EpicsMotor('XF:31ID-ES{Dif:Nano-Ax:TopX}Mtr', name='nano_top_x')
```
see also:

https://blueskyproject.io/ophyd/generated/ophyd.epics_motor.EpicsMotor.html#:~:text=class%20ophyd.epics_motor.EpicsMotor%28prefix%3D%27%27%2C%20%2A%2C%20name%2C%20kind%3DNone%2C%20read_attrs%3DNone%2C%20configuration_attrs%3DNone%2C%20parent%3DNone%2C,are%20passed%20through%20to%20the%20base%20class%2C%20Positioner




In [22]:
from ophyd import EpicsMotor
import ophyd
ophyd.set_cl('caproto')

sample_x_mm = ophyd.EpicsMotor('bl531_xps2:sample_x_mm', name='sample stage horizontal')
sample_y_mm = ophyd.EpicsMotor('bl531_xps2:sample_y_mm', name='sample stage vertical')
beamstop_diode = ophyd.EpicsSignal('bl201-beamstop:current', name='beamstop diode')

#this works
sample_x_mm.wait_for_connection(timeout=2.0)
sample_x_mm.read()
sample_y_mm.wait_for_connection(timeout=2.0)
sample_y_mm.read()
beamstop_diode.wait_for_connection(timeout=2.0)
beamstop_diode.read()

{'beamstop diode': {'value': -0.8894, 'timestamp': 1730411765.611748}}

In [7]:
from bluesky.plans import scan
# just a motor scan, no detectors
RE(scan([], sample_x_mm, 1, 2, sample_y_mm, 1,2, 11))

#works, but does a weird scan
#https://nsls-ii.github.io/bluesky/generated/bluesky.plans.grid_scan.html#bluesky.plans.grid_scan


('07ac858a-1945-48c2-b1cc-a0099c5176a6',)

In [7]:
from bluesky.plans import grid_scan

RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f1183da2-0559-4b45-9fac-280258be189d',)

In [23]:
from bluesky.plans import grid_scan

RE(grid_scan([beamstop_diode], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('2b5c81a9-69b7-4261-bc74-2f1ab96253ba',)

In [24]:
header = db[-1]
header.table()

,time,sample stage horizontal,sample stage horizontal_user_setpoint,sample stage vertical,sample stage vertical_user_setpoint,beamstop diode
seq_num,,,,,,
1,2024-10-31 21:56:34.844740868,1.0,1.0,1.0,1.0,-0.8904
2,2024-10-31 21:56:36.335853338,1.0,1.0,1.5,1.5,-0.8896
3,2024-10-31 21:56:37.833946705,1.0,1.0,2.0,2.0,-0.8896
4,2024-10-31 21:56:40.015880346,1.5,1.5,1.0,1.0,-0.8902
5,2024-10-31 21:56:41.502079487,1.5,1.5,1.5,1.5,-0.8936
6,2024-10-31 21:56:42.987761974,1.5,1.5,2.0,2.0,-0.8945
7,2024-10-31 21:56:45.159467697,2.0,2.0,1.0,1.0,-0.8947
8,2024-10-31 21:56:46.655962467,2.0,2.0,1.5,1.5,-0.8929
9,2024-10-31 21:56:48.146364450,2.0,2.0,2.0,2.0,-0.8918


In [17]:
# some issues here -- not critical
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
# Send all metadata/data captured to the BestEffortCallback.
bec_token = RE.subscribe(bec)
# Make plots update live while scans run.
from bluesky.utils import install_kicker


/usr/local/lib/python3.10/dist-packages/bluesky/utils/__init__.py:934: UserWarning: bluesky.utils.install_qt_kicker is no longer necessary and has no effect. Please remove your use of it. It may be removed in a future release of bluesky.
  warnings.warn("bluesky.utils.install_qt_kicker is no longer necessary and "


In [21]:
RE.unsubscribe(bec_token)
RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f59ba064-15f1-4e77-a008-19e88331a6fc',)

In [ ]:
RE(scan(dets, motors, 25, 28, 61))